Dosyaları 1.5 snlik dosyalara ayırma

In [2]:
from pydub import AudioSegment
import os

# MP3 dosyalarının bulunduğu dizin
mp3_dir = r"C:\Users\sebah\voice\Data\sesVeriseti"
# WAV dosyalarının kaydedileceği dizin
wav_dir = r"C:\Users\sebah\voice\Data\wav verileri"

# MP3 dosyalarının listesi
mp3_files = os.listdir(mp3_dir)

# Her MP3 dosyasını işleyin
for mp3_file in mp3_files:
    # MP3 dosyasını yükle
    audio = AudioSegment.from_mp3(os.path.join(mp3_dir, mp3_file))
    # 1.5 saniyelik parçalara böl
    chunks = audio[::1000]
    
    # Parçaları kaydet
    for i, chunk in enumerate(chunks):
        chunk.export(os.path.join(wav_dir, f"{mp3_file[:-4]}_{i}.wav"), format="wav")

MFCC özelliklerini çıkarmak

Veri Ön İşleme ve Özellik Çıkarımı

In [5]:
import os
import librosa
import numpy as np

# Ses dosyalarının bulunduğu dizin
ses_dizin = r'C:\Users\sebah\voice\Data\wav verileri'

# MFCC özelliklerinin kaydedileceği dizin
mfcc_dizin = r'C:\Users\sebah\voice\Data\MFCC'

# MFCC parametreleri
n_mfcc = 40

# Framelere ayırma parametreleri
    # frame_length = 25  # milisaniye cinsinden
    # frame_stride = 10   # milisaniye cinsinden
hop_length = int(frame_stride * sr / 1000)  # Kayma uzunluğu
n_fft = int(frame_length * sr / 1000)  # FFT uzunluğu


# Dosyaları işleme
for dosya_adı in os.listdir(ses_dizin):
    if dosya_adı.endswith('.wav'):
        dosya_yolu = os.path.join(ses_dizin, dosya_adı)
        ses, sr = librosa.load(dosya_yolu, sr=None)
        mfcc = librosa.feature.mfcc(y=ses, sr=sr, n_mfcc=n_mfcc, hop_length=int(frame_stride * sr / 1000),
                                     n_fft=int(frame_length * sr / 1000))
        mfcc_dosya_adı = dosya_adı.split('.')[0] + '.npy'
        mfcc_dosya_yolu = os.path.join(mfcc_dizin, mfcc_dosya_adı)
        np.save(mfcc_dosya_yolu, mfcc)

print("MFCC özellikleri başarıyla oluşturuldu.")
print(mfcc.shape)

c:\Users\sebah\voice\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=800 is too large for input signal of length=512
  warnings.warn(
c:\Users\sebah\voice\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=800 is too large for input signal of length=640
  warnings.warn(
c:\Users\sebah\voice\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=800 is too large for input signal of length=128
  warnings.warn(
c:\Users\sebah\voice\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=800 is too large for input signal of length=384
  warnings.warn(
c:\Users\sebah\voice\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=800 is too large for input signal of length=768
  warnings.warn(
c:\Users\sebah\voice\venv\lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=800 is too large for input signal of length=256
  warnings.warn(
c:\Users\sebah\voice\venv\lib\site-packages\librosa\

MFCC özellikleri başarıyla oluşturuldu.
(40, 83)


Modeli Eğitmek

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import os
import joblib

# MFCC özelliklerinin bulunduğu dizin
mfcc_dizin = r'C:\Users\sebah\voice\Data\MFCC'

X = []
y = []

# MFCC dosyalarını yükleme ve etiket çıkarma
for dosya_adı in os.listdir(mfcc_dizin):
    if dosya_adı.endswith('.npy'):
        dosya_yolu = os.path.join(mfcc_dizin, dosya_adı)
        mfcc = np.load(dosya_yolu)
        X.append(np.mean(mfcc, axis=1))  # Ortalama MFCC vektörü
        etiket = dosya_adı.split('_')[0]  # Etiketi dosya adından çıkar
        y.append(etiket)
        print(f"Dosya: {dosya_adı}, Etiket: {etiket}")

X = np.array(X)
y = np.array(y)

# Etiketleri sayısal değerlere dönüştürme
le = LabelEncoder()
y = le.fit_transform(y)

# Veri kümesini eğitim ve test kümelerine ayırma
X_egitim, X_test, y_egitim, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Normalizasyon
scaler = StandardScaler()
X_egitim = scaler.fit_transform(X_egitim)
X_test = scaler.transform(X_test)

# MLP modeli oluşturma ve eğitme
model = MLPClassifier(
    hidden_layer_sizes=(100, 50),  # Daha karmaşık bir yapı
    activation='relu',             # ReLU aktivasyon fonksiyonu
    solver='adam',                 # Adam optimizasyon algoritması
    max_iter=1000,                 # Eğitim iterasyon sayısı
    random_state=42
)
model.fit(X_egitim, y_egitim)

# Modelin doğruluğunu değerlendirme
dogruluk = model.score(X_test, y_test)
print(f"Model doğruluğu: {dogruluk}")
print(f"Eğitim veri boyutu: {X_egitim.shape}")
print(f"Test veri boyutu: {X_test.shape}")

# Etiket dağılımını kontrol etme
from collections import Counter
print("Etiket Dağılımı:", Counter(y))


Dosya: common_voice_tr_36442116_0.npy, Etiket: common
Dosya: common_voice_tr_36442116_1.npy, Etiket: common
Dosya: common_voice_tr_36442116_2.npy, Etiket: common
Dosya: common_voice_tr_36442117_0.npy, Etiket: common
Dosya: common_voice_tr_36442117_1.npy, Etiket: common
Dosya: common_voice_tr_36442117_2.npy, Etiket: common
Dosya: common_voice_tr_36442119_0.npy, Etiket: common
Dosya: common_voice_tr_36442119_1.npy, Etiket: common
Dosya: common_voice_tr_36442119_2.npy, Etiket: common
Dosya: common_voice_tr_36442120_0.npy, Etiket: common
Dosya: common_voice_tr_36442120_1.npy, Etiket: common
Dosya: common_voice_tr_36442120_2.npy, Etiket: common
Dosya: common_voice_tr_36442122_0.npy, Etiket: common
Dosya: common_voice_tr_36442122_1.npy, Etiket: common
Dosya: common_voice_tr_36442122_2.npy, Etiket: common
Dosya: common_voice_tr_36442126_0.npy, Etiket: common
Dosya: common_voice_tr_36442126_1.npy, Etiket: common
Dosya: common_voice_tr_36442126_2.npy, Etiket: common
Dosya: common_voice_tr_36442

In [8]:
# Modeli diske kaydetme
model_kayit_yolu = r'C:\Users\sebah\voice\Data\Model.pkl'
joblib.dump(model, model_kayit_yolu)

['C:\\Users\\sebah\\voice\\Data\\Model.pkl']

In [9]:
from sklearn.metrics import classification_report, confusion_matrix

# Test veri seti üzerinde tahmin yapma
tahminler = model.predict(X_test)

# Sınıflandırma raporu ve karışıklık matrisini yazdırma
print("Sınıflandırma Raporu:")
print(classification_report(y_test, tahminler))

print("Karışıklık Matrisi:")
print(confusion_matrix(y_test, tahminler))

Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3915
           1       0.50      0.33      0.40         3
           2       0.40      0.50      0.44         4
           3       0.50      0.33      0.40         3
           4       0.00      0.00      0.00         3
           5       0.50      0.20      0.29         5
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         5
           8       0.50      0.25      0.33         4
           9       0.25      0.33      0.29         3
          10       0.00      0.00      0.00         3
          11       0.00      0.00      0.00         2
          12       0.25      0.50      0.33         2
          13       0.50      1.00      0.67         4
          14       0.50      0.33      0.40         3
          15       0.08      0.20      0.12         5
          16       0.50      0.25      0.33         4
     

c:\Users\sebah\voice\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sebah\voice\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sebah\voice\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Uygulama Geliştirme - Dosya

In [13]:
import numpy as np
import os
import joblib

# Eğitilmiş modeli yükleme
model_kayit_yolu = r'C:\Users\sebah\voice\Data\Model.pkl'
model = joblib.load(model_kayit_yolu)

# Test edilecek ses dosyasının MFCC özelliklerini yükleme
test_ses_dosyasi = r'C:\Users\sebah\voice\Data\MFCC\sebo73_8.npy'
mfcc = np.load(test_ses_dosyasi)

# Model üzerinden tahmin yapma
tahmin = model.predict(np.mean(mfcc, axis=1).reshape(1, -1))

# Tahmini sonuç
print("Tahmin edilen kişi: ", tahmin)

Tahmin edilen kişi:  [0]


Uygulama Geliştirme - Mikrofon

In [14]:
import numpy as np
import sounddevice as sd
import joblib
import librosa
import soundfile as sf
import os

# Eğitilmiş modeli yükleme
model_kayit_yolu = r'C:\Users\sebah\voice\Data\Model.pkl'
model = joblib.load(model_kayit_yolu)

sinif_isimleri = ['Sebahattin Tegi','Hüseyin Tasgeldi','Eren Öztürk','Arda Yemci']

# Mikrofondan ses almak için gerekli parametreler
saniye_basina_ornek = 44100  # Örnekleme hızı (örneğin, 44100 Hz)
saniye = 5  # 5 saniyelik ses al
kanal_sayisi = 1  # Tek kanallı ses

while True:
    print("Konuşun...")
    ses = sd.rec(int(saniye_basina_ornek * saniye), samplerate=saniye_basina_ornek, channels=kanal_sayisi, dtype='float32')
    sd.wait()  # Ses alımının tamamlanmasını bekleyin
    
    # Ses dosyasını WAV olarak kaydetme
    kayit_yolu = "kayit.wav"
    sf.write(kayit_yolu, np.squeeze(ses), saniye_basina_ornek)
    
    # WAV dosyasını yükleme ve MFCC özelliklerini çıkarma
    y, sr = librosa.load(kayit_yolu, sr=saniye_basina_ornek)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=128)
    mfcc = np.mean(mfcc.T, axis=0)  # Ortalama MFCC vektörü
    
    # Model üzerinden tahmin yapma
    tahmin_indeksi = model.predict(mfcc.reshape(1, -1))[0]
    tahmin_isim = sinif_isimleri[tahmin_indeksi]
    
    # Tahmini sonuç
    print("Tahmin edilen kişi: ", tahmin_isim)
    #os.remove(kayit_yolu)  # Kayıt dosyasını temizleme

Konuşun...


ValueError: X has 128 features, but MLPClassifier is expecting 40 features as input.

In [15]:
import speech_recognition as sr
from pydub import AudioSegment

def convert_mp3_to_wav(mp3_file_path, wav_file_path):
    audio = AudioSegment.from_mp3(mp3_file_path)
    audio.export(wav_file_path, format="wav")

def transcribe_audio(audio_file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file_path) as source:
        audio = recognizer.record(source)
    try:
        transcript = recognizer.recognize_google(audio, language="tr-TR")
        return transcript
    except sr.UnknownValueError:
        return "Could not understand audio"
    except sr.RequestError as e:
        return f"Could not request results; {e}"

kelimeler = []


if __name__ == "__main__":
    wav_file_path = "kayit.wav"  # Path to the converted WAV file

    # Transcribe the WAV file-
    transcript = transcribe_audio(wav_file_path)
    
    kelimeler.extend(transcript.split())
    
    print("Transcript:")
    print(transcript)
    
    print("Kelime Sayısı:")
    print(len(kelimeler))

Transcript:
esselamün aleyküm ve rahmetullah
Kelime Sayısı:
4
